# SYMFLUENCE Tutorial 02b — Basin-Scale Workflow (Bow River at Banff, Semi-Distributed)

## Introduction

This tutorial advances from lumped to semi-distributed watershed modeling. Instead of representing the basin as a single unit, we now subdivide it into multiple connected sub-basins (GRUs) that capture spatial variability while maintaining computational efficiency.

Building on Tutorial 02a's lumped approach, semi-distributed modeling adds spatial detail through automated watershed delineation that creates multiple sub-basins, stream network topology that connects GRUs through routing, and spatially-distributed characteristics that better represent elevation gradients and heterogeneous processes.

The key configuration change is `DOMAIN_DEFINITION_METHOD: 'delineate'` with a `STREAM_THRESHOLD` parameter controlling the number of sub-basins. Smaller thresholds create more GRUs (finer spatial detail) but increase computational cost.

We continue with the **Bow River at Banff** watershed, now discretized into multiple GRUs connected by mizuRoute for explicit stream network routing. This approach improves representation of snowmelt timing, spatial climate variability, and runoff generation patterns.


# Step 1 — Configuration and data reuse

We generate a semi-distributed configuration and intelligently reuse data from Tutorial 02a where possible.

In [ ]:
# Step 1 — Semi-distributed configuration with data reuse

from pathlib import Path
import shutil
from symfluence import SYMFLUENCE
from symfluence.core.config.models import SymfluenceConfig

config = SymfluenceConfig.from_minimal(
    # Basic identification
    domain_name='Bow_at_Banff_semi_distributed',
    experiment_id='run_1',

    # Gauging station coordinates (Banff WSC 05BB001)
    pour_point_coords='51.1722/-115.5717',

    # Semi-distributed basin settings
    definition_method='delineate',
    DELINEATION_METHOD='stream_threshold',
    STREAM_THRESHOLD=5000,
    discretization='GRUs',

    # Model configuration
    model='SUMMA',
    routing_model='mizuRoute',
    MIZU_FROM_MODEL='SUMMA',

    # mizuRoute routing configuration
    SETTINGS_MIZU_ROUTING_VAR='averageRoutedRunoff',
    SETTINGS_MIZU_ROUTING_UNITS='m/s',
    SETTINGS_MIZU_ROUTING_DT='3600',
    SETTINGS_MIZU_OUTPUT_VARS=1,

    # Temporal extent
    time_start='2004-01-01 01:00',
    time_end='2007-12-31 23:00',
    calibration_period='2005-10-01, 2006-09-30',
    evaluation_period='2006-10-01, 2007-12-30',
    spinup_period='2004-01-01, 2005-09-30',

    # Streamflow observations
    station_id='05BB001',
    DOWNLOAD_WSC_DATA=True,

    # Calibration settings
    OPTIMIZATION_METHODS=['iteration'],
    params_to_calibrate='minStomatalResistance,cond2photo_slope,vcmax25_canopyTop,jmax25_scale,summerLAI,rootingDepth,soilStressParam,z0Canopy,windReductionParam',
    optimization_target='streamflow',
    optimization_algorithm='DDS',
    optimization_metric='KGE',
    calibration_timestep='hourly',
)

# Save configuration
import yaml
config_path = Path('./config_semi_distributed.yaml')
config_dict = config.to_dict(flatten=True)
with open(config_path, 'w') as f:
    yaml.dump(config_dict, f, default_flow_style=False, sort_keys=False)
print(f"Configuration saved: {config_path}")

# === Data reuse from Tutorial 02a ===
SYMFLUENCE_DATA_DIR = Path(str(config.system.data_dir)).resolve()
print(f"Using SYMFLUENCE_DATA_DIR: {SYMFLUENCE_DATA_DIR}")

lumped_domain = 'Bow_at_Banff_lumped'
lumped_data_dir = SYMFLUENCE_DATA_DIR / f'domain_{lumped_domain}'

def copy_with_name_adaptation(src, dst, old_name, new_name):
    """Copy directory and adapt filenames"""
    if not src.exists():
        return False
    dst.parent.mkdir(parents=True, exist_ok=True)
    if src.is_file():
        shutil.copy2(src, dst)
        return True
    shutil.copytree(src, dst, dirs_exist_ok=True)
    # Rename files containing old domain name
    for file in dst.rglob('*'):
        if file.is_file() and old_name in file.name:
            new_file = file.parent / file.name.replace(old_name, new_name)
            file.rename(new_file)
    return True

# Initialize SYMFLUENCE with visualization enabled
symfluence = SYMFLUENCE(config, visualize=True)
project_dir = symfluence.managers['project'].setup_project()

if lumped_data_dir.exists():
    print(f"\nReusing data from Tutorial 02a: {lumped_data_dir}")
    
    reusable_data = {
        'Elevation': lumped_data_dir / 'attributes' / 'elevation',
        'Land Cover': lumped_data_dir / 'attributes' / 'landclass',
        'Soils': lumped_data_dir / 'attributes' / 'soilclass',
        'Forcing': lumped_data_dir / 'forcing' / 'raw_data',
        'Streamflow': lumped_data_dir / 'observations' / 'streamflow'
    }
    
    for data_type, src_path in reusable_data.items():
        if src_path.exists():
            rel_path = src_path.relative_to(lumped_data_dir)
            dst_path = project_dir / rel_path
            success = copy_with_name_adaptation(src_path, dst_path, lumped_domain, config.domain.name)
            if success:
                print(f"   {data_type}: Copied")
        else:
            print(f"   {data_type}: Not found")
else:
    print(f"\nNo data from Tutorial 02a found at: {lumped_data_dir}")
    print(f"   Will acquire fresh data.")

# Create pour point
pour_point_path = symfluence.managers['project'].create_pour_point()
print(f"\nProject structure created at: {project_dir}")

## Step 2 — Domain definition (multi-GRU)

Delineate the watershed into multiple sub-basins using stream network analysis and create connected GRUs.

### Step 2a — Attribute check

Verify DEM availability from data reuse, or acquire fresh if needed.

In [ ]:
# Step 2a — DEM availability check
dem_path = project_dir / 'attributes' / 'elevation' / 'dem'
if not dem_path.exists() or len(list(dem_path.glob('*.tif'))) == 0:
    print("   DEM not found, acquiring geospatial attributes...")
    # Uncomment to acquire data (set DATA_ACCESS to 'cloud' or 'maf' in config)
    # symfluence.managers['data'].acquire_attributes()
    print("✅ Geospatial attributes acquired")
else:
    print("✅ DEM available from previous workflow")

### Step 2b — Stream network delineation

Automated watershed subdivision based on stream threshold parameter.

In [ ]:
# Step 2b — Stream network delineation
watershed_path = symfluence.managers['domain'].define_domain()
print("✅ Stream network delineation complete")

### Step 2c — GRU discretization

Convert sub-basins to GRUs with routing connectivity.

In [ ]:
# Step 2c — GRU discretization
hru_path = symfluence.managers['domain'].discretize_domain()
print("✅ GRU discretization complete")

### Step 2d — Network visualization

Visualize the semi-distributed structure: sub-basins and stream network.

In [ ]:
# Step 2d — Network structure visualization (using native SYMFLUENCE plotting)

from IPython.display import Image, display

# Generate native domain visualization (includes GRUs and stream network)
plot_path = symfluence.managers['domain'].visualize_discretized_domain()

if plot_path:
    print(f"Domain plot saved to: {plot_path}")
    display(Image(filename=str(plot_path)))
else:
    print("Domain visualization not generated. Check logs for errors.")

## Step 3 — Data preprocessing

Process forcing and observation data for multiple GRUs.

In [ ]:
# Step 3a — Streamflow observations
# Uncomment to download and process observations
# symfluence.managers['data'].process_observed_data()
print("✅ Streamflow data processing complete")

In [ ]:
# Step 3b — Forcing data
# Uncomment to acquire data (set DATA_ACCESS to 'cloud' or 'maf' in config)
# symfluence.managers['data'].acquire_forcings()
print("✅ Forcing acquisition complete")

In [ ]:
# Step 3c — Model-agnostic preprocessing
symfluence.managers['data'].run_model_agnostic_preprocessing()
print("✅ Model-agnostic preprocessing complete")

## Step 4 — Model execution

Configure and run SUMMA-mizuRoute with multiple connected GRUs.

In [ ]:
# Step 4a — Model configuration
symfluence.managers['model'].preprocess_models()
print("✅ Semi-distributed model configuration complete")

In [ ]:
# Step 4b — Model execution
symfluence.managers['model'].run_models()
print("✅ Semi-distributed simulation complete")

## Step 5 — Evaluation

Compare semi-distributed results against observations.

In [ ]:
# Step 5 — Semi-distributed evaluation (using Camille's model comparison plots)

from IPython.display import Image, display

# Generate model comparison overview (auto-detects mizuRoute outputs)
plot_path = symfluence.managers['reporting'].generate_model_comparison_overview(
    experiment_id=config.domain.experiment_id,
    context='run_model'
)

if plot_path:
    print(f"Model comparison overview: {plot_path}")
    display(Image(filename=str(plot_path)))
else:
    print("No model outputs found for comparison. Check simulation outputs.")

print("\nSemi-distributed evaluation complete")

# Step 5b — Run calibration 



In [ ]:
results_file = symfluence.managers['optimization'].calibrate_model()  
print("Calibration results file:", results_file)

In [ ]:
# Step 5c — Post-calibration visualization 

from IPython.display import Image, display

# Generate post-calibration visualizations
plot_paths = symfluence.managers['reporting'].visualize_calibration_results(
    experiment_id=config.domain.experiment_id
)

# Display all generated plots
for plot_name, plot_path in plot_paths.items():
    print(f"\n{plot_name}:")
    display(Image(filename=str(plot_path)))

print("\nPost-calibration visualization complete")